In [1]:
import torch
from torch_geometric.nn import knn_graph, knn


import numpy as np
from tqdm import tqdm

import os
import glob
import re

# sys.path.append('../Utility Scripts')

from Utility_functions import print_3D_graph, manual_print_3D_graph

import params

print('NSIM:', params.NSIM)
print('NSTEP:', params.NSTEP)
print('FLUID_PROBES:', params.NFLUID_PROBES)
print('IO_PROBES:', params.NIO_PROBES)
print('NPOS:', params.NPOS)
print('NFEATURES:', params.NFEATURES)
print('DATADIR:', params.DATADIR)

NSIM: 3
NSTEP: 7000
FLUID_PROBES: 3643
IO_PROBES: 0
NPOS: 6
NFEATURES: 10
DATADIR: ../.data/Extracted_data


In [2]:

# test_sim000 = np.load("../.data/Extracted_data/probes/SIM000_probepos.npy")

# print(len(test_sim000))
# print(test_sim000) 

In [3]:

# test_sim000 = np.load("../.data/Extracted_data/fields/SIM000_features.npz")

# print(test_sim000.files)

# for name in test_sim000.files:
#     print(f"{name}:")
#     print(test_sim000[name].tolist())
#     print(test_sim000[name].shape)
#     print(len(test_sim000[name]))
#     print("\n" + "-"*50 + "\n")  

# READING NPZ AND NPY

In [4]:
numbers = re.compile(r'(\d+)')

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [5]:
probes_path = os.path.join(params.DATADIR, 'probes')
probes = sorted(glob.glob(os.path.join(probes_path, '*.npy')), key = numericalSort)
print(probes[0])

fields_path = os.path.join(params.DATADIR, 'fields')
fields = sorted(glob.glob(os.path.join(fields_path, '*.npz')), key = numericalSort)
print(fields[0])

../.data/Extracted_data\probes\SIM001_probepos.npy
../.data/Extracted_data\fields\SIM001_features.npz


In [6]:
# probe_data_list = []

# for data_file in probes:
#     read_points = np.load(data_file)
#     probe_data_list.append(torch.from_numpy(read_points))

# pos_list = probe_data_list
# #print(pos_list)
# print(len(pos_list))
# print(pos_list[0].tolist())
# print(len(pos_list[0]))
# print(pos_list[0])


In [7]:
features_list = []

for data_file in fields:
    read_features = np.load(data_file)
    num_points = None
    for _, filedat in enumerate(read_features.files):
        if num_points is None:
            num_points = len(read_features[filedat])
            # print(len(read_features[filedat]))
            feature_tensor = torch.zeros(num_points, 10)
            # print(len(feature_tensor))
        #print(read_features[filedat])
        feature_tensor[:, _] = torch.from_numpy(read_features[filedat]).squeeze()
    features_list.append(feature_tensor)

feat_list = features_list
print(feat_list[0])
print(len(feat_list))

tensor([[ 1.0573e+05,  1.5147e-24, -1.5929e-31,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0573e+05,  1.2055e-12, -1.5435e-13,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0573e+05, -8.7818e-13, -2.0277e-13,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.0575e+05, -7.4240e-03, -3.0406e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0573e+05, -4.9733e-03,  5.6077e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0573e+05, -4.9733e-03,  5.6077e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
3


In [8]:
print(len(probes))
assert params.NSIM == len(probes)

dataset_len = len(probes)  # number of simulations
print(probes)
positions = np.empty((params.NSIM, params.NPOS, params.NFLUID_PROBES + params.NIO_PROBES))
#print(positions)
for i, data in enumerate(probes):
    read_points = np.load(data)
    #print(read_points)
    positions[i,:] = read_points.T
    
pos = torch.from_numpy(np.einsum('ijk->ikj', positions))
print(len(pos))
print(pos[0])
#print(pos[1])
print(pos.shape) # [params.NSIM, params.NFLUID_PROBES + params.NIO_PROBES, params.NPOS]

3
['../.data/Extracted_data\\probes\\SIM001_probepos.npy', '../.data/Extracted_data\\probes\\SIM002_probepos.npy', '../.data/Extracted_data\\probes\\SIM003_probepos.npy']
3
tensor([[ -2.0775,   0.0325,  15.7272,   0.0000,   0.0000,   0.0000],
        [ -2.0718,   0.3807,  15.7272,   0.0000,   0.0000,   0.0000],
        [ -1.4163,   0.3999,  15.7861,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.7945,  -1.2432, -17.3447,   0.0000,   0.0000,   0.0000],
        [  0.8598,  -0.3687, -20.9809,   0.0000,   0.0000,   0.0000],
        [  0.9352,  -0.4223, -20.5151,   0.0000,   0.0000,   0.0000]],
       dtype=torch.float64)
torch.Size([3, 3643, 6])


In [9]:
# features = np.empty((params.NSIM, params.NSIM * NUM_FEATURES, NUM_NODES))
####positions = np.empty((params.NSIM, params.NPOS, params.NFLUID_PROBES + params.NIO_PROBES))
features = np.zeros((params.NSIM, params.NFRAME * params.NFEATURES, params.NFLUID_PROBES + params.NIO_PROBES))
feat = features.reshape((params.NSIM, params.NFRAME, params.NFEATURES, params.NFLUID_PROBES + params.NIO_PROBES))
# print(features.shape)
# #print(feat)
# print('Features shape:', features.shape)     # [1000, 60 (6*10), 2000]
#print(fields)
for i, data in enumerate(tqdm(fields)):

    read_features = np.load(data)
    print(read_features.files)
    print(read_features)
    for j, filedat in enumerate(read_features.files): 
        # print(i, j, filedat, read_features[filedat].shape, features[i, j, :].shape)
        #features[i, j, :params.NFLUID_PROBES] = read_features[filedat]
        features[i, j, :] = read_features[filedat]
        #print(features[i, j, :params.NFLUID_PROBES])

#print(features[0][0].tolist())

100%|██████████| 3/3 [00:00<00:00, 499.82it/s]

['p', 'U_x', 'U_y', 'U_z', 'T', 'rho', 'phi', 'zero1', 'zero2', 'zero3']
NpzFile '../.data/Extracted_data\\fields\\SIM001_features.npz' with keys: p, U_x, U_y, U_z, T...
['p', 'U_x', 'U_y', 'U_z', 'T', 'rho', 'phi', 'zero1', 'zero2', 'zero3']
NpzFile '../.data/Extracted_data\\fields\\SIM002_features.npz' with keys: p, U_x, U_y, U_z, T...
['p', 'U_x', 'U_y', 'U_z', 'T', 'rho', 'phi', 'zero1', 'zero2', 'zero3']
NpzFile '../.data/Extracted_data\\fields\\SIM003_features.npz' with keys: p, U_x, U_y, U_z, T...


In [10]:
#print(features[0][1].tolist()) # vel_x
feat = features.reshape((params.NSIM, params.NFRAME, params.NFEATURES, params.NFLUID_PROBES + params.NIO_PROBES))

feat = torch.from_numpy(np.einsum('ijkl->ijlk', feat))
print(feat.shape)
print(feat[0][0].tolist()) 
#print(len(feat[0][0]))

torch.Size([3, 1, 3643, 10])
[[105729.0, 1.51468e-24, -1.59291e-31, 30.0, 300.0, 1.225, 147.0, 0.0, 0.0, 0.0], [105729.0, 1.20546e-12, -1.54351e-13, 30.0, 300.0, 1.225, 147.006, 0.0, 0.0, 0.0], [105729.0, -8.78178e-13, -2.02771e-13, 30.0, 300.0, 1.225, -0.00367496, 0.0, 0.0, 0.0], [105729.0, 4.93432e-15, -8.40678e-15, 30.0, 300.0, 1.225, 0.000734963, 0.0, 0.0, 0.0], [105729.0, 8.68201e-13, -4.52456e-13, 30.0, 300.0, 1.225, 4.54747e-13, 0.0, 0.0, 0.0], [105729.0, -1.63172e-10, 1.06499e-09, 30.0, 300.0, 1.225, -0.0146998, 0.0, 0.0, 0.0], [105729.0, -7.13603e-08, -1.01674e-07, 30.0, 300.0, 1.225, 0.00257261, 0.0, 0.0, 0.0], [105729.0, -5.28074e-08, -1.61503e-06, 30.0, 300.0, 1.225, 0.00735, 0.0, 0.0, 0.0], [105729.0, 1.63806e-06, 1.17201e-05, 30.0, 300.0, 1.225, 147.008, 0.0, 0.0, 0.0], [105729.0, -4.2621e-10, -1.26621e-07, 30.0, 300.0, 1.225, -0.0220353, 0.0, 0.0, 0.0], [105729.0, -4.07833e-07, 5.38249e-07, 30.0, 300.0, 1.225, 147.003, 0.0, 0.0, 0.0], [105729.0, -4.36492e-15, -8.68018e-1

In [11]:
# new_pos_list = [pos.unsqueeze(0).repeat(params.NFRAME, 1, 1) for pos in pos_list]
# print('Node position and labels data:', [pos.shape for pos in new_pos_list])
# print('Fluid dynamic features   data:', [feat.shape for feat in feat_list])

# dataset = [torch.cat([new_pos, feat.unsqueeze(0)], dim=-1) for new_pos, feat in zip(new_pos_list, feat_list)]

# print('Concatenated total data:', [data.shape for data in dataset])


In [12]:
new_pos = pos.unsqueeze(1).repeat(1, params.NFRAME, 1, 1)
print('Node position and labels data:', new_pos.shape)
print('Fluid dynamic features   data:', feat.shape)

dataset = torch.cat([new_pos, feat], dim = -1)
print(len(dataset))
print(dataset[0][0][0])

print('Concatenated total data:', dataset.shape) # [100,6,2100,16]

Node position and labels data: torch.Size([3, 1, 3643, 6])
Fluid dynamic features   data: torch.Size([3, 1, 3643, 10])
3
tensor([-2.0775e+00,  3.2520e-02,  1.5727e+01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0573e+05,  1.5147e-24, -1.5929e-31,  3.0000e+01,
         3.0000e+02,  1.2250e+00,  1.4700e+02,  0.0000e+00,  0.0000e+00,
         0.0000e+00], dtype=torch.float64)
Concatenated total data: torch.Size([3, 1, 3643, 16])


In [13]:
# cut to last frame, we are interested in the steady state

print('Data with all time frames:', dataset.shape)     # [1000,6,2100,16] [samples,frames, nodes, tot_features]

new_dataset = dataset[:, -1:, :, :].squeeze(1) # only last frame
print(new_dataset[0][0]) # coord_x, coord_y, coord_z, mis, size, sdf, press, vel_x, vel_y, vel_z, stress_ij

print('Data with only last frame:', new_dataset.shape)    # [samples, 2100, 16]

Data with all time frames: torch.Size([3, 1, 3643, 16])
tensor([-2.0775e+00,  3.2520e-02,  1.5727e+01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0573e+05,  1.5147e-24, -1.5929e-31,  3.0000e+01,
         3.0000e+02,  1.2250e+00,  1.4700e+02,  0.0000e+00,  0.0000e+00,
         0.0000e+00], dtype=torch.float64)
Data with only last frame: torch.Size([3, 3643, 16])


# PREPROCESSING OF MIS VALUES

In [14]:
# # clean up MIS to have smooth values next to wall (due to the staircase-like discretization)

# spacing = params.SPACING
# kneigh_shells = params.NEIGHBOURS

# idx = 0
# nodes = new_dataset.shape[1]
# # edge_index = knn_graph(new_dataset[idx,:,:3], params.NEIGHBOURS)
# edge_index = torch.tensor(np.load(params.DATADIR+"/connectivity.npy"))
# # print('knn_graph:', edge_index.shape)
# print(edge_index.shape)
# print(edge_index)

# for idx in range(0):
#     print_3D_graph(new_dataset[idx,:,:3], edge_index, color = new_dataset[idx,:,4]) # original

# for data in tqdm(new_dataset):

#     print(data)
# #     # selecting near-surface nodes (outer shell)
# #     idx_shell1 = (data[:,3] <= spacing)
# #     pos_shell1 = data[idx_shell1,:3]

# #     # selecting near-to-outer shell nodes
# #     idx_shell2 = (data[:,3]>spacing)*(data[:,3]<=3*spacing)
# #     pos_shell2 = data[idx_shell2,:3]

# #     # ordered array of the two shells' nodes
# #     shells_pos = torch.cat((pos_shell1,pos_shell2),0)

# #     # creating edges between set1 and set2
# #     edge_indexes = knn(pos_shell2,pos_shell1, kneigh_shells)

# #     # storing indexes of edge-receiving nodes
# #     edge_reshaped = edge_indexes.reshape(-1)[int(sum(idx_shell1)*kneigh_shells):]

# #     # get the MIS value for the edge-receiving nodes
# #     sdf_reshaped = data[idx_shell2][edge_reshaped][:,4].reshape(-1,kneigh_shells)

# #     def most_frequent(row):
# #         i = 0
# #         values, counts = np.unique(row, return_counts=True)
# #         ind = np.argmax(counts)
# #         return values[ind]

# #     # for every row, calculate the most occurring MIS value
# #     # the i-th row represents the i-th edge-sending node (nodes belonging to the outer shell)
# #     new_val = np.apply_along_axis(most_frequent, 1, sdf_reshaped)

# #     # change outer shell's MIS values with the ones computed now
# #     data[idx_shell1,4] = torch.tensor(new_val)

# #     # indexing on the whole set of points has to be coherent with nodes ordering 
# #     # (knn function with two sets of nodes, orders both indexing from 0)
# #     edge_indexes_plot = edge_indexes.clone()
# #     edge_indexes_plot[1,:] += edge_indexes_plot[0,-1]+1


# # print_3D_graph(new_dataset[idx,:,:3], edge_index, color = new_dataset[idx,:,4]) # processed

# CHOKED NODES VISUALIZATION

In [15]:

# chocked_nodes = torch.tensor(np.load(params.DATADIR+"/chocked_nodes.npy"))
# print(f"chocked_nodes: {chocked_nodes}")

# node_list = [0] * (params.NFLUID_PROBES+1)
# print(len(chocked_nodes))
# for index in chocked_nodes:
#     node_list[index] = 1

# # node_list[48] = 0.3
# # node_list[49] = 0.3
# # node_list[100] = 0.7
# # node_list[101] = 0.7

# print(f"node_list: {node_list}")
# print(len(node_list))

# manual_print_3D_graph(new_dataset[idx,:,:3], edge_index, color = node_list, colorscale="Greens")


# READ A GRAPH AND CONSTRUCT EDGE CONNECTIVITY

In [16]:
# check if given the big tensor storing all simulations, I am able to recover a specific steady state and its quantities

for i in range(1):
    single_graph = new_dataset[i,...]

    # i=0
    # while i < 61:#len(single_graph):
    #     print(f"{i}th node: {torch.norm(single_graph[i,7:10])}")
    #     i += 1

    neighbours = params.NEIGHBOURS

    # print(single_graph[1]) # coord_x, coord_y, coord_z, mis, sdf, size, press, vel_x, vel_y, vel_z, stress_ij

    for i in range(3):
        print(single_graph[i, 7:10].tolist())

    #magvel = torch.norm(single_graph[..., 4:7], dim=-1)
    magvel = torch.norm(single_graph[..., 7:10], dim=-1) # 7:10 are vel_x, vel_y, vel_z
    print(f"magvel: {magvel}")   

    print('Single graph:', single_graph.shape)
    # print('Edge index  :', edge_index.shape)
    print('Velocity magnitude:', magvel.shape)

    print(" ")
    print("VELOCITY:")
    print_3D_graph(single_graph, None, magvel)

    # [(0, -1), (0, 1), (1, 2), (1, 4), (2, 3),
    # (4, 5), (3, 6), (3, 8), (5, 10), (5, 12), 
    # (6, 7), (8, 9), (10, 11), (12, 13)]

[1.51468e-24, -1.59291e-31, 30.0]
[1.20546e-12, -1.54351e-13, 30.0]
[-8.78178e-13, -2.02771e-13, 30.0]
magvel: tensor([30.0000, 30.0000, 30.0000,  ..., 29.9957, 29.9993, 29.9993],
       dtype=torch.float64)
Single graph: torch.Size([3643, 16])
Velocity magnitude: torch.Size([3643])
 
VELOCITY:


# READ PRESSURE

In [17]:

single_graph = new_dataset[0,...]

press = single_graph[..., 6] # 6th column is pressure
print(press.tolist())
print(" ")
print("PRESSURE:")
print_3D_graph(single_graph, None, press)


[105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105690.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 106116.0, 105732.0, 105741.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105731.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105777.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105727.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105678.0, 105729.0, 105689.0, 105695.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0, 105729.0,

# CHECKING MIS PREPROCESSING

In [18]:
# idx = 0 # steady state idx
# neighbours = params.NEIGHBOURS
# nodes = new_dataset.shape[1]
# new_dataset_copy = new_dataset.clone()

# # print(new_dataset_copy.shape)

# print_3D_graph(new_dataset_copy[idx,:,:3], edge_index, color = new_dataset_copy[idx,:,4]) # original    
# print_3D_graph(new_dataset[idx,:,:3], edge_index, color = new_dataset[idx,:,4]) # processed

In [19]:
# counts = []

# # checking number of neighbors per node
# for i in range(params.NFLUID_PROBES + params.NIO_PROBES):
#     count = (edge_index[1,:] == i)
#     count = count.sum()
#     counts.append(count)

# print('edge_index[:10]', edge_index[:10])
# print('counts[:10]:', counts[:10])

# count = torch.tensor(counts, dtype=torch.float32)
# print('Mean:', count.mean(), 'Std:', count.std())

# SAVING DATASET

In [20]:
saved_dataset = new_dataset.reshape(-1, params.NFLUID_PROBES + params.NIO_PROBES, params.NFEATURES + 6).float()


In [21]:
raw_path = os.path.join(params.DATADIR, 'raw')
processed_path = os.path.join(params.DATADIR, 'processed')
dataset_name = 'Dataset.pt'

if not os.path.exists(raw_path):
    os.makedirs(raw_path)

if not os.path.exists(processed_path):
    os.makedirs(processed_path)

torch.save(saved_dataset, os.path.join(raw_path, dataset_name))